In [1]:
CUDA_LAUNCH_BLOCKING=1

In [2]:
import os
if os.getcwd().endswith('notebooks'):
    os.chdir("..")
    
print(f'Current working directory: {os.getcwd()}')

Current working directory: /home/robert/Workspace/university/thesis-public


In [3]:
!pip install -e .

Obtaining file:///home/robert/Workspace/university/thesis-public
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
You should consider upgrading via the '/home/robert/tools/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
from src.preprocess.model_specific.contrastive import ContrastivePreprocessorKnownClusters, \
    ContrastivePreprocessorUnknownClusters
from src.preprocess.model_specific.word_cooc import WordCoocPreprocessor
from src.preprocess.standardize import RelationalDatasetStandardizer, WDCDatasetStandardizer, CSVNoSplitStandardizer
from src.utils import seed_all

seed_all(42)

stand_path = 'configs/stands_tasks/proprietary_scarce.json'
standardizer = CSVNoSplitStandardizer(stand_path)
standardizer.preprocess()

proc_path = 'configs/model_specific/contrastive/proprietary_scarce.json'
preprocessor = ContrastivePreprocessorUnknownClusters(proc_path)

target_split = standardizer.config.target_location.split('/')
directory = target_split[-1] if target_split[-1] != '' else target_split[-2]
default_preproc_target = os.path.join('data', 'processed', 'contrastive', directory)

preprocessor.preprocess(original_location=standardizer.config.target_location,
                        target_location=default_preproc_target)

All files are already in place, skipping step for target location: data/interim/proprietary_scarce/
All files are already in place, skipping step for target location: data/processed/contrastive/proprietary_scarce


In [5]:
import pandas as pd
from typing import Tuple

def load_data(path: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    return pd.read_csv(os.path.join(path, 'train.csv')),\
            pd.read_csv(os.path.join(path, 'valid.csv')),\
            pd.read_csv(os.path.join(path, 'test.csv'))

In [6]:
train_set, valid_set, test_set = load_data('data/processed/contrastive/proprietary_scarce')

In [12]:
from src.predictors.contrastive import ContrastivePredictor
from src.predictors.cross_encoders import DittoPredictor
from src.preprocess.configs import ExperimentsArgumentParser

predictor = ContrastivePredictor(config_path='configs/model_train/contrastive/frozen_no-aug_batch-pt128_proprietary-scarce.json')

arguments = ExperimentsArgumentParser()
arguments.parse_args("")
arguments.load_wandb_models = True
predictor.train(train_set, valid_set, arguments=arguments)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D/resolve/main/config.json from cache at /home/robert/.cache/huggingface/transformers/740f3ffeb47194ea97e78563af6f51d8fce8d8346d36d41889c75780869cf609.71ad48e6ad371e0c3ffa4982a5cc1c97e41023678e120b71ebe50addd42ec567
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 3

Successfully loaded model from path: ./artifacts/model-278nalif:v0/pytorch_model.bin


In [13]:
f1 = predictor.test(test_set=test_set)
print(f'F1 score of loaded model: {f1}')

***** Running Prediction *****
  Num examples = 613
  Batch size = 8


F1 score of loaded model: 0.4196428571428572


In [14]:
f1 = predictor.test(test_set=valid_set)
print(f'F1 score on the validation set of the loaded model: {f1}')

***** Running Prediction *****
  Num examples = 612
  Batch size = 8


F1 score on the validation set of the loaded model: 0.39449541284403666


In [16]:
f1 = predictor.test(test_set=train_set)
print(f'F1 score on the train set of the loaded model: {f1}')

***** Running Prediction *****
  Num examples = 3474
  Batch size = 8


F1 score on the train set of the loaded model: 0.7770177838577291


In [37]:
test_set = train_set

In [38]:
from src.predictors.base import ClassificationDataset

test_dataset = ClassificationDataset(test_set)
label_pred = predictor.trainer.predict(test_dataset)
label_pred = [v[0] for v in label_pred.predictions]
label_true = test_set['label'].tolist()

diff = [0 if label_pred[i] != label_true[i] else 1 for i in range(len(label_pred))]
print(diff)

***** Running Prediction *****
  Num examples = 1578
  Batch size = 8


[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 

In [39]:
false_negatives = [i for i in range(len(diff)) if diff[i] == 0 and label_pred[i] == 0]
false_positives = [i for i in range(len(diff)) if diff[i] == 0 and label_pred[i] == 1]

print(false_negatives)
print(false_positives)

print(len(false_negatives))
print(len(false_positives))
print(len(test_set))
print(len(train_set))

[0, 20, 47, 77, 78, 88, 129, 133, 153, 172, 192, 198, 215, 216, 227, 249, 264, 270, 280, 314, 324, 326, 333, 379, 383, 463, 479]
[]
27
0
1578
1578


In [40]:
import random
# random_sample = random.sample(range(1, len(test_set)), 100)

random_sample = false_negatives

In [41]:
for i in range(len(random_sample)):
    print(f'Index: {random_sample[i]}')
    print(f'Text left: {test_set.iloc[random_sample[i]]["left_text"]}')
    print(f'Text right: {test_set.iloc[random_sample[i]]["right_text"]}')
    print(f'True label: {test_set.iloc[random_sample[i]]["label"]}')
    print(f'Predicted label: {label_pred[random_sample[i]]}')
    print('===============================================')

Index: 0
Text left: [COL] title [VAL] asus prime x299 deluxe prijzen tweakers [COL] description [VAL] nan [COL] specTableContent [VAL] categorie moederborden merk asus product asus prime x299 deluxe uitvoering asus prime x299 deluxe ean 4712900763881 sku 90mb0ty0 m0eay0 prime x299 deluxe tweakers id 776903 productprijs 398 95 pakketpost nederland 0 ideal 0 totaal 398 95 onjuiste prijs melden productprijs 395 pakketpost 3 95 ideal 0 totaal 398 95 onjuiste prijs melden productprijs 399 verzending 0 ideal 0 totaal 399 onjuiste prijs melden productprijs 399 pakketpost tot 50 0 ideal 0 totaal 399 onjuiste prijs melden productprijs 409 pakketpost vanaf 50 0 ideal 0 totaal 409 onjuiste prijs melden productprijs 409 pakketpost vanaf 25 0 ideal 0 totaal 409 onjuiste prijs melden productprijs 421 pakketpost nederland vanaf 75 0 ideal 0 totaal 421 onjuiste prijs melden productprijs 428 22 opsturen vanaf 50 0 ideal 0 totaal 428 22 onjuiste prijs melden productprijs 427 51 pakketpost dpd 3 95 ideal

In [61]:
raw_test = pd.read_csv('data/raw/amazon_google/test.csv')
raw_left = pd.read_csv('data/raw/amazon_google/tableA.csv')
raw_right = pd.read_csv('data/raw/amazon_google/tableB.csv')

In [68]:
raw_test.iloc[1022]

ltable_id     369
rtable_id    3007
label           0
Name: 1022, dtype: int64

In [63]:
print(raw_left.iloc[369])
print(raw_right.iloc[3007])

id                                                          369
title           tlc arthur 's kindergarten learning system 2008
manufacturer                                             encore
price                                                     19.99
Name: 369, dtype: object
id                                                    3007
title           reader rabbit reading learning system 2007
manufacturer                                           NaN
price                                                12.95
Name: 3007, dtype: object


In [66]:
processed_test = pd.read_csv('data/processed/contrastive/amazon_google/test.csv')

In [69]:
processed_test.iloc[1022]

left_text     [COL] title [VAL] videostudio 11 plus [COL] ma...
right_text    [COL] title [VAL] video studio 11 plus [COL] m...
label                                                         0
Name: 1022, dtype: object

In [22]:
interim_test = pd.read_csv('data/interim/amazon_google/test.csv')

In [23]:
interim_test.iloc[1085]

left_id                                         572
left_title            adobe dreamweaver cs3 upgrade
left_manufacturer                             adobe
right_id                                        954
right_title           adobe dreamweaver cs3 upgrade
right_manufacturer                              NaN
label                                             0
Name: 1085, dtype: object

In [75]:
raw_test[(raw_test['ltable_id'] == 572) & (raw_test['rtable_id'] == 954)]

,ltable_id,rtable_id,label
958,572,954,0


In [76]:
print(raw_left[raw_left['id'] == 572])
print(raw_right[raw_right['id'] == 954])

      id                          title manufacturer  price
572  572  adobe dreamweaver cs3 upgrade        adobe  199.0
      id                          title manufacturer   price
954  954  adobe dreamweaver cs3 upgrade          NaN  205.99


TODO: 
- extract problematic labels as csv to export to google sheets
- how much are the problematic labels affecting my f1 score?



In [19]:
mislabeled = [1022, 1085, 1151, 1201, 1289, 1328, 1390, 1505, 1665, 1782, 1818, 1896, 2026, 2053, 2060, 2061, 2121, 2152, 2156, 2165, 2246, 2270, 2271, 2291, 2088]
doubts = [1223, 1412, 1525, 1572, 1961, 2052, 2073, 2118, 2210, 1944, 1289]

mislabeled_test_set = test_set[test_set.index.isin(mislabeled)]
doubts_test_set = test_set[test_set.index.isin(doubts)]

print(len(mislabeled_test_set))
print(len(doubts_test_set))

25
11


In [32]:
mislabeled_interim = pd.merge(interim_test, mislabeled_test_set, how='inner', left_index=True, right_index=True)
mislabeled_interim.drop(['left_text', 'right_text', 'label_y'], axis=1, inplace=True)
mislabeled_interim.head()

,left_id,left_title,left_manufacturer,right_id,right_title,right_manufacturer,label_x
1022,856,videostudio 11 plus,corel,5,video studio 11 plus,corel corporation,0
1085,572,adobe dreamweaver cs3 upgrade,adobe,954,adobe dreamweaver cs3 upgrade,NaN,0
1151,1029,westward,encore software,2081,encore software 13781 westward,NaN,0
1201,831,rainbow fish and the whale ( win/mac ),global-software-publishing,2970,rainbow fish and the whale,NaN,0
1289,1334,adobe creative suite cs3 web standard,adobe,566,adobe cs3 web standard,NaN,0


In [36]:
import os

if not os.path.exists('output/data/amazong_google'):
    os.makedirs('output/data/amazon_google')

mislabeled_interim.to_csv('output/data/amazon_google/mislabeled.csv')

In [40]:
doubts_interim = pd.merge(interim_test, doubts_test_set, how='inner', left_index=True, right_index=True)
doubts_interim.drop(['left_text', 'right_text', 'label_y'], axis=1, inplace=True)
doubts_interim.to_csv('output/data/amazon_google/doubts.csv')

In [48]:
from sklearn.metrics import f1_score

f1 = f1_score(label_true, label_pred)
print(f'F1 score with original labels: {f1}')

label_corrected = label_true.copy()
for i in mislabeled: 
    label_corrected[i] = (label_true[i] + 1) % 2
    
f1_corrected = f1_score(label_corrected, label_pred)
print(f'F1 score with corrected labels: {f1_corrected}')

F1 score with original labels: 0.842741935483871
F1 score with corrected labels: 0.8944337811900192
